In [ ]:
import pandas as pd
import numpy as np
import mne
import matplotlib.pyplot as plt
from mne.stats import permutation_cluster_test


#导入要比较的数据
c_data = cache.read(rf"{CACHE_DIR}\avg_all_patient_level_power\C_{brain_zone}_delta")
i_data = cache.read(rf"{CACHE_DIR}\avg_all_patient_level_power\I_{brain_zone}_delta")

cmp_data = []
real_data = []
for i in range(c_data.shape[0]):
    real_data.append(i_data[i] - c_data[i])
    cmp_data.append(np.zeros(c_data[i].shape))
cmp_data = np.mean(np.array(cmp_data), axis=1)
real_data = np.mean(np.array(real_data), axis=1)

times = []
for i in range(2001):
    times.append(-0.5 + 1 / 1000 * i)
print(cmp_data.shape)
T_obs, clusters, cluster_p_values, H0 = \
    permutation_cluster_test([cmp_data, real_data], tail=0, out_type='mask',
                             n_permutations=1000)
print(enumerate(clusters))
for i_c, c in enumerate(clusters):
    c = c[0]

    if cluster_p_values[i_c] <= 0.05 and times[c.start]>0:
        h = ax.axvspan(times[c.start], times[c.stop - 1],
                       color='gray', alpha=0.3,label="cluster p-value < 0.05" if i_c == 0 else "" )
        print({brain_zone}, times[c.start], times[c.stop - 1],'T_obs',T_obs[i_c],'cluster_p_values',cluster_p_values[i_c])
ax.axvline(0,color='dimgray',linestyle='--',linewidth=5)
ax.set_xlim(-0.5, 1.5)
#ax.set_xlim(-1.5, 0.5)
ax.set(xlabel='Time (s)', ylabel=f'Delta power',
       title=f"{brain_zone}")
plt.show()
#fig.savefig(f"./CueAlignedavgpower_patient_level/ALLpatient_{brain_zone}_{task_type_list[0]}_{task_type_list[1]}_delta_off.png",dpi=300)
